In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib inline
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan 
import numpy as np
import ld_gan.utils.utils as ld
from time import time
from tqdm import tqdm

# load vgg

In [3]:
vgg = ld_gan.models.VGG()

# load data

In [4]:
X, Y = ld_gan.data_proc.data_loader.load_data(11, resize=64)
Y = np.argmax(Y, axis = 1)
n_classes = int(Y.max() + 1)

100%|██████████| 102/102 [00:10<00:00, 13.57it/s]

load data from '/export/home/oblum/projects/ld_gan/data/flowers_102/jpg_test_256'


In [5]:
X1 = X[:50]
X2 = X[50:]

# compute features

In [ ]:
def apply_vgg(vgg, imgs, batch_size = None):
    
    imgs = ld_gan.data_proc.norm_img.norm(imgs)
    
    if batch_size is None:
        x = ld_gan.data_proc.transformer.np_to_tensor(imgs, normalize=False)
        C1, C2, C3, C4, C5 = vgg(x)
        C1, C2, C3, C4, C5 = ld_gan.data_proc.transform(C1, C2, C3, C4, C5,
                                                        normalize=False)
        
    else:
        n_iters = int(imgs.shape[0] / batch_size) + 1
        X = np.array_split(imgs, n_iters)
        C1, C2, C3, C4, C5 = [], [], [], [], []
        for x in tqdm(X):
            x = ld_gan.data_proc.transformer.np_to_tensor(x, normalize=False)
            c1, c2, c3, c4, c5 = vgg(x)
            c1, c2, c3, c4, c5 = ld_gan.data_proc.transform(c1, c2, c3, c4, c5,
                                                            normalize=False)
            C1.append(c1)
            C2.append(c2)
            C3.append(c3)
            C4.append(c4)
            C5.append(c5)
        C1 = np.concatenate(C1)
        C2 = np.concatenate(C2)
        C3 = np.concatenate(C3)
        C4 = np.concatenate(C4)
        C5 = np.concatenate(C5)

    return imgs, C1, C2, C3, C4, C5

def nn_vgg(vgg, imgs_all, imgs_batch, n_neighbors=1, batch_size=16):
    
    C0, C1, C2, C3, C4, C5 = apply_vgg(vgg, imgs_all, batch_size)
    
    dists = []
    idxs = []
    for img in tqdm(imgs_batch):
        c0, c1, c2, c3, c4, c5 = apply_vgg(vgg, np.array([img]))
        
        err = np.apply_over_axes(np.mean, np.abs(C0-c0), (1,2,3)) + \
              np.apply_over_axes(np.mean, np.abs(C1-c1), (1,2,3)) + \
              np.apply_over_axes(np.mean, np.abs(C2-c2), (1,2,3)) + \
              np.apply_over_axes(np.mean, np.abs(C3-c3), (1,2,3)) + \
              np.apply_over_axes(np.mean, np.abs(C4-c4), (1,2,3)) + \
              np.apply_over_axes(np.mean, np.abs(C5-c5), (1,2,3))
        err = np.squeeze(err)
        
        idxs_img = np.argsort(err)[:n_neighbors]
        
        dists.append(err[idxs_img])
        idxs.append(idxs_img)
        
    return dists, idxs

In [ ]:
dists, idxs = nn_vgg(vgg, X2, X1)

  0%|          | 0/50 [00:00<?, ?it/s]